In [1]:
!pip install -U -qq openai langchain chromadb faiss-cpu pypdf tiktoken docarray langchain-community langchain-openai sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.3 MB/s eta 0:

In [2]:
# Import the required libraries
import pandas as pd
import os
import openai
import numpy as np
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAI
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder, util
from langchain.prompts import PromptTemplate

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Set the API key
filepath = "/content/drive/My Drive/GenAI/" # Change filepath as required. Nothing else needs to change

with open(filepath + "OPENAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

os.environ["OPENAI_API_KEY"] = openai.api_key

In [5]:
# Read the email details file
dfDetails = pd.read_csv(filepath + 'email_thread_details.csv')
dfDetails.head()


,thread_id,subject,timestamp,from,to,body
0,1,FW: Master Termination Log,2002-01-29 11:23:42,"Gossett, Jeffrey C. JGOSSET","['Giron', 'Darron C. Dgiron', 'Love', 'Phillip...",\n\n -----Original Message-----\nFrom: =09Ther...
1,1,FW: Master Termination Log,2002-01-31 12:50:00,"Theriot, Kim S. KTHERIO","['Murphy', 'Melissa Mmurphy', 'Gossett', 'Jeff...",\n\n -----Original Message-----\nFrom: =09Panu...
2,1,FW: Master Termination Log,2002-02-05 15:03:35,"Theriot, Kim S. KTHERIO","['Murphy', 'Melissa Mmurphy', 'Anderson', 'Dia...",Note to Stephanie Panus....\n\nStephanie...ple...
3,1,FW: Master Termination Log,2002-02-05 15:06:25,"Theriot, Kim S. KTHERIO","['Hall', 'D. Todd Thall', 'Sweeney', 'Kevin Ks...",\n\n -----Original Message-----\nFrom: =09Panu...
4,1,FW: Master Termination Log,2002-05-28 07:20:35,"Kelly, Katherine L. KKELLY","['Germany', 'Chris Cgerman']",\n\n -----Original Message-----\nFrom: =09McMi...


In [6]:
# Read the email summary file
dfSummary = pd.read_csv(filepath + 'email_thread_summaries.csv')
dfSummary.head()

,thread_id,summary
0,1,The email thread discusses the Master Terminat...
1,2,A lunch meeting has been scheduled for May 5th...
2,3,Ben is updating a friend on his progress with ...
3,4,The recipient of the email thread initially ex...
4,5,The email thread discusses the long form confi...


In [7]:
# Remove the "Original Message" from body of emails since that is repeating text
dfDetails[['body_modified', 'body_modified_2']] = dfDetails['body'].str.split('-----Original Message-----', n=1, expand=True)
dfDetails.head()

,thread_id,subject,timestamp,from,to,body,body_modified,body_modified_2
0,1,FW: Master Termination Log,2002-01-29 11:23:42,"Gossett, Jeffrey C. JGOSSET","['Giron', 'Darron C. Dgiron', 'Love', 'Phillip...",\n\n -----Original Message-----\nFrom: =09Ther...,\n\n,"\nFrom: =09Theriot, Kim S. =20\nSent:=09Tuesda..."
1,1,FW: Master Termination Log,2002-01-31 12:50:00,"Theriot, Kim S. KTHERIO","['Murphy', 'Melissa Mmurphy', 'Gossett', 'Jeff...",\n\n -----Original Message-----\nFrom: =09Panu...,\n\n,"\nFrom: =09Panus, Stephanie =20\nSent:=09Thurs..."
2,1,FW: Master Termination Log,2002-02-05 15:03:35,"Theriot, Kim S. KTHERIO","['Murphy', 'Melissa Mmurphy', 'Anderson', 'Dia...",Note to Stephanie Panus....\n\nStephanie...ple...,Note to Stephanie Panus....\n\nStephanie...ple...,"\nFrom: =09Panus, Stephanie =20\nSent:=09Tuesd..."
3,1,FW: Master Termination Log,2002-02-05 15:06:25,"Theriot, Kim S. KTHERIO","['Hall', 'D. Todd Thall', 'Sweeney', 'Kevin Ks...",\n\n -----Original Message-----\nFrom: =09Panu...,\n\n,"\nFrom: =09Panus, Stephanie =20\nSent:=09Tuesd..."
4,1,FW: Master Termination Log,2002-05-28 07:20:35,"Kelly, Katherine L. KKELLY","['Germany', 'Chris Cgerman']",\n\n -----Original Message-----\nFrom: =09McMi...,\n\n,"\nFrom: =09McMichael Jr., Ed =20\nSent:=09Tues..."


In [8]:
# Remove the new lines characters
dfDetails['body_modified'] = dfDetails['body_modified'].str.replace('\n',' ')

In [9]:
# Create the message by combining timestamp, from, to and body
dfDetails['message'] = "Subject: " + dfDetails['subject'] + " ; Time Stamp: " + dfDetails['timestamp'] + " ; From: " + dfDetails['from'] + " ; To: " + dfDetails['to'] + " ; Body: " + dfDetails['body_modified']

In [10]:
# Drop the columns that are not required
dfDetails.drop(columns=['timestamp', 'from', 'to', 'body', 'body_modified', 'body_modified_2'], axis = 0, inplace = True)

In [11]:
# Join the two tables
dfSummary['id'] = dfSummary['thread_id']
dfSummary.set_index('thread_id', inplace=True)
dfSummary['message'] = dfDetails.groupby('thread_id')['message'].apply(' '.join)

dfSummary.head()

,summary,id,message
thread_id,,,
1,The email thread discusses the Master Terminat...,1,Subject: FW: Master Termination Log ; Time Sta...
2,A lunch meeting has been scheduled for May 5th...,2,Subject: Credit Group Lunch ; Time Stamp: 2000...
3,Ben is updating a friend on his progress with ...,3,Subject: New Address ; Time Stamp: 2000-01-09 ...
4,The recipient of the email thread initially ex...,4,Subject: EOL Data ; Time Stamp: 2001-02-01 09:...
5,The email thread discusses the long form confi...,5,Subject: RE: long form confirm/MDEA ; Time Sta...


In [12]:
# Create the subject column for metadata. Add id to the subject since chroma db is not accepting ids separately
dfSummary['subject'] = dfSummary['message'].str.split(';', n=1).str[0]
dfSummary['subject'] = dfSummary['subject'].apply(lambda x: x.split(':')[-1])
dfSummary['subject'] = dfSummary['subject'] + ":" + dfSummary['id'].astype(str)
dfSummary.head()

dfSummary

,summary,id,message,subject
thread_id,,,,
1,The email thread discusses the Master Terminat...,1,Subject: FW: Master Termination Log ; Time Sta...,Master Termination Log :1
2,A lunch meeting has been scheduled for May 5th...,2,Subject: Credit Group Lunch ; Time Stamp: 2000...,Credit Group Lunch :2
3,Ben is updating a friend on his progress with ...,3,Subject: New Address ; Time Stamp: 2000-01-09 ...,New Address :3
4,The recipient of the email thread initially ex...,4,Subject: EOL Data ; Time Stamp: 2001-02-01 09:...,EOL Data :4
5,The email thread discusses the long form confi...,5,Subject: RE: long form confirm/MDEA ; Time Sta...,long form confirm/MDEA :5
...,...,...,...,...
4163,Peter Thompson has sent a memo to Kay Mann and...,4163,Subject: ltr to Kay Mann: Site specific refere...,Site specific references in GE Form agreement...
4164,The email thread revolves around the sharing a...,4164,Subject: presentation ; Time Stamp: 2000-10-09...,presentation :4164
4165,Susan asks Emily about her plans for the weeke...,4165,Subject: this weekend ; Time Stamp: 2001-09-27...,this weekend :4165


In [13]:
# Just use the required columns - summary and subject
df = dfSummary[["summary", 'subject']]
df

,summary,subject
thread_id,,
1,The email thread discusses the Master Terminat...,Master Termination Log :1
2,A lunch meeting has been scheduled for May 5th...,Credit Group Lunch :2
3,Ben is updating a friend on his progress with ...,New Address :3
4,The recipient of the email thread initially ex...,EOL Data :4
5,The email thread discusses the long form confi...,long form confirm/MDEA :5
...,...,...
4163,Peter Thompson has sent a memo to Kay Mann and...,Site specific references in GE Form agreement...
4164,The email thread revolves around the sharing a...,presentation :4164
4165,Susan asks Emily about her plans for the weeke...,this weekend :4165


**Generate and Store Embeddings using OpenAI and ChromaDB**

In [14]:
# Load the data from dataframe to langchain
loader = DataFrameLoader(df, page_content_column="summary")
email_data = loader.load()

In [15]:
email_data[0]

Document(metadata={'subject': ' Master Termination Log :1'}, page_content="The email thread discusses the Master Termination Log and the need to investigate a CNG LDC (Hope Gas) termination and a $66 million settlement offer. Stephanie Panus sends out the Daily List and Master Termination Log for various dates. Kim Theriot requests her name and Melissa Murphy's name to be removed from the distribution list and adds several names to it. The thread also includes updates on terminations and valid terminations for various companies.")

In [16]:
# Store the split document in ChromaDB
db = Chroma.from_documents(email_data, OpenAIEmbeddings())

<ipython-input-16-2ae6f9e5f79f>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  db = Chroma.from_documents(email_data, OpenAIEmbeddings())


In [17]:
query = "Tell me more about Enron 2001: An inside view"

docs = db.similarity_search(query)

print(docs[0].page_content)

Global Change Associates is promoting a corporate profile titled "Enron 2001: An Inside View" that provides insight into Enron's operations and success. The study covers topics such as the firm's structure, employee recruitment and training, risk-free transactions, negotiation strategies, and forgiveness of counterparty obligations. The publication is available for purchase at $995. Some recipients express interest in obtaining a copy, while others question the credibility of the report and discuss potential actions to prevent its marketing.


**Semantic Search**



In [18]:
query = input()

When did Jeff Dasovich inform the group that he faxed PG&E's filing to Rick Shapiro?


In [19]:
# Chroma db retriever
retriever = db.as_retriever()

In [20]:
# Get upto 5 documents from the vector store
results = retriever.get_relevant_documents(query, k = 5)
results

<ipython-input-20-277729191f7e>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  results = retriever.get_relevant_documents(query, k = 5)


[Document(metadata={'subject': " Response Statement to PG&E's Ch. 11 Filing :1081"}, page_content="Enron's Jeff Dasovich informs the group that he has faxed PG&E's filing to Rick Shapiro, stating that the filing is brief and lists the top 20 creditors, with ENE Canada being the only Enron listing. He also mentions that the realignment of rates could have prevented PG&E's bankruptcy. Eric Thode responds to a call from the AP, providing a statement and refusing further comments on Enron's exposure and opinions on Governor Davis' speech. Karen Denne expresses disappointment in PG&E's bankruptcy and criticizes the lack of leadership in addressing the energy crisis. She also states that Enron's exposure to PG&E will not significantly affect their earnings or financial condition."),
 Document(metadata={'subject': ' Please Take a Look--Draft Slides for Monday Talk in New York on\n California Crisis :3020'}, page_content='Jeff Dasovich sent an email to several recipients, including Richard Sha

In [21]:
# Create a dataframe from the results; as shown by the instructor
results_df = pd.DataFrame()

for i in range(len(results)):
  result_dict = results[i].dict()
  df_temp = pd.DataFrame.from_dict(result_dict)
  results_df = pd.concat([results_df, df_temp], ignore_index=True)

results_df


,id,metadata,page_content,type
0,None,Response Statement to PG&E's Ch. 11 Filing :1081,Enron's Jeff Dasovich informs the group that h...,Document
1,None,Please Take a Look--Draft Slides for Monday T...,Jeff Dasovich sent an email to several recipie...,Document
2,None,Press Statement on Bad PUC Gas Decision? Need...,The email thread discusses a press statement o...,Document
3,None,Hertzberg :124,Jeff Dasovich forwards a powerpoint presentati...,Document


**Re-ranking the Results**

In [22]:
# Initialise the cross encoder model
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
# Rerank as shown by the instructor
cross_inputs = [[query, response] for response in results_df['page_content']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)
results_df['Reranked_scores'] = cross_rerank_scores
results_df

,id,metadata,page_content,type,Reranked_scores
0,None,Response Statement to PG&E's Ch. 11 Filing :1081,Enron's Jeff Dasovich informs the group that h...,Document,7.372217
1,None,Please Take a Look--Draft Slides for Monday T...,Jeff Dasovich sent an email to several recipie...,Document,0.579248
2,None,Press Statement on Bad PUC Gas Decision? Need...,The email thread discusses a press statement o...,Document,-3.710442
3,None,Hertzberg :124,Jeff Dasovich forwards a powerpoint presentati...,Document,1.378114


In [24]:
# Separate the id
results_df['id'] = results_df['metadata'].str.split(':', n=1).str[1]
results_df

,id,metadata,page_content,type,Reranked_scores
0,1081,Response Statement to PG&E's Ch. 11 Filing :1081,Enron's Jeff Dasovich informs the group that h...,Document,7.372217
1,3020,Please Take a Look--Draft Slides for Monday T...,Jeff Dasovich sent an email to several recipie...,Document,0.579248
2,520,Press Statement on Bad PUC Gas Decision? Need...,The email thread discusses a press statement o...,Document,-3.710442
3,124,Hertzberg :124,Jeff Dasovich forwards a powerpoint presentati...,Document,1.378114


In [25]:
# Add the message column from details to results. The message is a summary of the emails
j = 0
col_list = results_df['id'].tolist()
results_df['Message'] = ""
for id in col_list:
  if (not np.isnan(float(id))):
    results_df['Message'][j] = dfSummary['message'][int(id)]
  j = j + 1

results_df

<ipython-input-25-c0fe23a18d3b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Message'][j] = dfSummary['message'][int(id)]
<ipython-input-25-c0fe23a18d3b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Message'][j] = dfSummary['message'][int(id)]
<ipython-input-25-c0fe23a18d3b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Message'][j] = dfSummary['message'][int(id)]
<ipython-input-25-c0fe23a

,id,metadata,page_content,type,Reranked_scores,Message
0,1081,Response Statement to PG&E's Ch. 11 Filing :1081,Enron's Jeff Dasovich informs the group that h...,Document,7.372217,Subject: Response Statement to PG&E's Ch. 11 F...
1,3020,Please Take a Look--Draft Slides for Monday T...,Jeff Dasovich sent an email to several recipie...,Document,0.579248,Subject: Please Take a Look--Draft Slides for ...
2,520,Press Statement on Bad PUC Gas Decision? Need...,The email thread discusses a press statement o...,Document,-3.710442,Subject: Press Statement on Bad PUC Gas Decisi...
3,124,Hertzberg :124,Jeff Dasovich forwards a powerpoint presentati...,Document,1.378114,Subject: Hertzberg ; Time Stamp: 2001-03-12 10...


In [26]:
# Return the top 3 results from semantic search
top_3_rank = results_df.sort_values(by='Reranked_scores', ascending = False)
top_3_rank[:3]

,id,metadata,page_content,type,Reranked_scores,Message
0,1081,Response Statement to PG&E's Ch. 11 Filing :1081,Enron's Jeff Dasovich informs the group that h...,Document,7.372217,Subject: Response Statement to PG&E's Ch. 11 F...
3,124,Hertzberg :124,Jeff Dasovich forwards a powerpoint presentati...,Document,1.378114,Subject: Hertzberg ; Time Stamp: 2001-03-12 10...
1,3020,Please Take a Look--Draft Slides for Monday T...,Jeff Dasovich sent an email to several recipie...,Document,0.579248,Subject: Please Take a Look--Draft Slides for ...


In [27]:
# Get the required information for top 3 results
top_3_RAG = top_3_rank[["page_content", "metadata", "Message"]][:3]
top_3_RAG

,page_content,metadata,Message
0,Enron's Jeff Dasovich informs the group that h...,Response Statement to PG&E's Ch. 11 Filing :1081,Subject: Response Statement to PG&E's Ch. 11 F...
3,Jeff Dasovich forwards a powerpoint presentati...,Hertzberg :124,Subject: Hertzberg ; Time Stamp: 2001-03-12 10...
1,Jeff Dasovich sent an email to several recipie...,Please Take a Look--Draft Slides for Monday T...,Subject: Please Take a Look--Draft Slides for ...


**Retrieval Augmented Generation**

In [28]:
delimiter = "####"
prompt_template = f"""

  You are a helpful assistant and your goal is to find useful information from the email content and email summary provided to you.

  {delimiter}
  Use the documents in '{top_3_RAG}' to answer the query '{query}'
  {delimiter}

  {delimiter}
  Here are some instructions around the email requirements
  To find the perfect response, look for more information in the provided data
  1. Document provides the summary of the email threads
  2. Metadata provides the subject of the email threads
  3. Email threads provide all the threads. The threads have timestamp, from, to and body. From field is the person sending the email and to field is the list of recipients
  4. Analyse the document, metadata and email threads to answer the query
  5. You are given 3 set of documents, metadata and email threads. Analyze all of them to answer the query
  6. Only answer the relevant information. If you do not have required information, please provide information that may help user to find the required response to the query
  7. You are an assistant. Do not provide any information on internal workings. Only provide final response.
  {delimiter}

  """

prompt = PromptTemplate(
    input_variables = ["top_3_RAG", "query"], template = prompt_template
)

In [29]:
# Initialize the LLM
llm = ChatOpenAI()

In [30]:
# Create a simple chain of providing a prompt and getting a response
chain = prompt | llm
chain.invoke({'top_3_RAG': top_3_RAG, "query": query}).content

"Jeff Dasovich informed the group that he faxed PG&E's filing to Rick Shapiro on March 12, 2001."